In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dense, Dropout, Input, Layer
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K


In [ ]:
def load_data(file_path):
    data = pd.read_csv(file_path)
    labels = data['label'].values
    features = data.drop(columns=['label', 'video_name']).values
    return features, labels

# Đọc dữ liệu từ các tệp CSV
X_train, y_train = load_data("dataset/train.csv")
X_test, y_test = load_data("dataset/test.csv")
X_val, y_val = load_data("dataset/validation.csv")

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

# Định hình lại dữ liệu để phù hợp với đầu vào của GRU
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
X_val = np.reshape(X_val, (X_val.shape[0], 1, X_val.shape[1]))

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)  
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

In [ ]:
# Định nghĩa lớp Attention
class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight', shape=(input_shape[-1], 1), initializer='random_normal', trainable=True)
        self.b = self.add_weight(name='attention_bias', shape=(input_shape[1], 1), initializer='zeros', trainable=True)
        super(Attention, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        output = x * a
        return K.sum(output, axis=1)

In [ ]:
# Xây dựng mô hình Bi-GRU với Attention
input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))
gru_layer = Bidirectional(GRU(128, return_sequences=True))(input_layer)
attention_layer = Attention()(gru_layer)
dropout_layer = Dropout(0.5)(attention_layer)
output_layer = Dense(1, activation='sigmoid')(dropout_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Huấn luyện mô hình
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Đánh giá mô hình
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')

# Dự đoán
predictions = model.predict(X_test)

# In kết quả dự đoán
for i in range(len(predictions)):
    print(f"Video {i+1}: Predicted Label: {np.argmax(predictions[i])}")


y_train shape: (114, 6)
y_test shape: (30, 6)


In [ ]:
predictions = model.predict(X_test)

# In kết quả dự đoán
for i in range(len(predictions)):
    print(f"Video {i+1}: Predicted Label: {np.argmax(predictions[i])}")
